# Variational AutoEncoder (MLP) Pytorch
Train a variational autoencoder with MNIST dataset

#### References:
* http://kvfrans.com/variational-autoencoders-explained/
* https://github.com/pytorch/examples/blob/master/mnist/main.py
* https://github.com/pytorch/examples/blob/master/vae/main.py
* http://wiseodd.github.io/techblog/2017/01/24/vae-pytorch/
* https://github.com/ritheshkumar95/pytorch-vqvae
* https://pytorch.org/docs/stable/distributions.html
* https://ermongroup.github.io/cs228-notes/extras/vae/

In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torch.distributions.normal import Normal
from torchvision import datasets, transforms
from torchvision.utils import save_image
#from torch.utils.tensorboard import SummaryWriter
batch_size = 128
ZDIMS = 10
num_hidden = 800
epochs = 10
num_classes = 10
print("PyTorch Version: ",torch.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)
num_gpu = torch.cuda.device_count()
print('Number of available GPUs:', num_gpu)

log_interval = 10

PyTorch Version:  1.0.0
Device: cpu
Number of available GPUs: 0


#### Download Data

In [2]:
# Download or load downloaded MNIST dataset
# shuffle data at every epoch
train_loader = torch.utils.data.DataLoader(datasets.MNIST('data', 
    train=True, download=True,transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

# Same for test data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=False)

#### Helper Function to convert Labels to One_Hot

In [3]:
def idx2onehot(idx, n=num_classes):

    assert idx.shape[1] == 1
    assert torch.max(idx).item() < n

    onehot = torch.zeros(idx.size(0), n)
    onehot.scatter_(1, idx.data, 1)

    return onehot

#### Define Model

In [4]:
class CondVAE(nn.Module):
    def __init__(self):
        super(CondVAE, self).__init__()
        # Encoder
        self.fc1 = nn.Linear((28*28)+10, num_hidden)
        self.fc20 = nn.Linear(num_hidden, num_hidden)
        self.fc21 = nn.Linear(num_hidden, ZDIMS)
        self.fc22 = nn.Linear(num_hidden, ZDIMS)
        
        # Decoder
        self.fc3 = nn.Linear(ZDIMS+10, num_hidden)
        self.fc31 = nn.Linear(num_hidden, num_hidden)
        self.fc4 = nn.Linear(num_hidden, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h1_2 = F.relu(self.fc20(h1))
        return self.fc21(h1_2), self.fc22(h1_2)

    def decode(self, z_concat):
        h3 = F.relu(self.fc3(z_concat))
        h31 = F.relu(self.fc31(h3))
        return torch.sigmoid(self.fc4(h31))
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        # Sample from normal 0 mean unit variance distribution
        eps = torch.randn_like(std)
        # Reparametrization trick
        guessed_z = mu + (eps*std)
        return guessed_z

    def forward(self, x, y_one_hot):
        # Concatenate image and labels
        x = x.view(-1, 784)
        x = torch.cat((x, y_one_hot), dim=1)
        # Encode
        mu, logvar = self.encode(x)
        # Reparametrization Trick
        z = self.reparameterize(mu, logvar)
        # Concatenate z with labels
        z = torch.cat((z, y_one_hot), dim=1)
        return self.decode(z), mu, logvar

#### Instantiate Model

In [5]:
model = CondVAE().to(device)

#### Define Optimizer

In [10]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

#### Define Loss Function

In [11]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

#### Define Train/Test Functions

In [12]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, label) in enumerate(train_loader):
        data = data.to(device)
        # convert y into one-hot encoding
        y = idx2onehot(label.view(-1, 1))
        y = y.to(device)
        
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data, y)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, label) in enumerate(test_loader):
            data = data.to(device)
            # convert y into one-hot encoding
            y = idx2onehot(label.view(-1, 1))
            y = y.to(device)
            recon_batch, mu, logvar = model(data, y)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

#### Train/Test

In [13]:
# 206 loss
for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)
        
        with torch.no_grad():
            z = torch.randn(64, ZDIMS).to(device)
            #y = torch.randint(0, num_classes, (64, 1)).to(dtype=torch.long)
            y = torch.ones(64, 1).to(dtype=torch.long) * 8

            y = idx2onehot(y).to(device, dtype=z.dtype)
            z = torch.cat((z, y), dim=1)

            sample = model.decode(z).cpu()
            save_image(sample.view(64, 1, 28, 28), 'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 98.395538
Train Epoch: 1 [1280/60000 (2%)]	Loss: 92.932953
Train Epoch: 1 [2560/60000 (4%)]	Loss: 91.132523
Train Epoch: 1 [3840/60000 (6%)]	Loss: 99.634056
Train Epoch: 1 [5120/60000 (9%)]	Loss: 96.466217
Train Epoch: 1 [6400/60000 (11%)]	Loss: 94.970703
Train Epoch: 1 [7680/60000 (13%)]	Loss: 98.552620
Train Epoch: 1 [8960/60000 (15%)]	Loss: 97.525772
Train Epoch: 1 [10240/60000 (17%)]	Loss: 96.834557
Train Epoch: 1 [11520/60000 (19%)]	Loss: 91.218605
Train Epoch: 1 [12800/60000 (21%)]	Loss: 95.728691
Train Epoch: 1 [14080/60000 (23%)]	Loss: 100.011978
Train Epoch: 1 [15360/60000 (26%)]	Loss: 97.763550
Train Epoch: 1 [16640/60000 (28%)]	Loss: 93.235291
Train Epoch: 1 [17920/60000 (30%)]	Loss: 99.700073
Train Epoch: 1 [19200/60000 (32%)]	Loss: 95.920258
Train Epoch: 1 [20480/60000 (34%)]	Loss: 94.763443
Train Epoch: 1 [21760/60000 (36%)]	Loss: 100.798637
Train Epoch: 1 [23040/60000 (38%)]	Loss: 94.691956
Train Epoch: 1 [24320/60000 (41%)]	Loss: 94.5

Train Epoch: 4 [21760/60000 (36%)]	Loss: 93.498863
Train Epoch: 4 [23040/60000 (38%)]	Loss: 97.387833
Train Epoch: 4 [24320/60000 (41%)]	Loss: 94.654144
Train Epoch: 4 [25600/60000 (43%)]	Loss: 93.688538
Train Epoch: 4 [26880/60000 (45%)]	Loss: 96.245285
Train Epoch: 4 [28160/60000 (47%)]	Loss: 99.084755
Train Epoch: 4 [29440/60000 (49%)]	Loss: 98.176193
Train Epoch: 4 [30720/60000 (51%)]	Loss: 93.191269
Train Epoch: 4 [32000/60000 (53%)]	Loss: 95.177170
Train Epoch: 4 [33280/60000 (55%)]	Loss: 92.210991
Train Epoch: 4 [34560/60000 (58%)]	Loss: 96.854156
Train Epoch: 4 [35840/60000 (60%)]	Loss: 95.154259
Train Epoch: 4 [37120/60000 (62%)]	Loss: 95.652313
Train Epoch: 4 [38400/60000 (64%)]	Loss: 93.610542
Train Epoch: 4 [39680/60000 (66%)]	Loss: 96.879730
Train Epoch: 4 [40960/60000 (68%)]	Loss: 95.443909
Train Epoch: 4 [42240/60000 (70%)]	Loss: 97.876099
Train Epoch: 4 [43520/60000 (72%)]	Loss: 94.060097
Train Epoch: 4 [44800/60000 (75%)]	Loss: 89.636536
Train Epoch: 4 [46080/60000 (77

Train Epoch: 7 [43520/60000 (72%)]	Loss: 95.425400
Train Epoch: 7 [44800/60000 (75%)]	Loss: 96.636993
Train Epoch: 7 [46080/60000 (77%)]	Loss: 96.045227
Train Epoch: 7 [47360/60000 (79%)]	Loss: 100.024841
Train Epoch: 7 [48640/60000 (81%)]	Loss: 94.325562
Train Epoch: 7 [49920/60000 (83%)]	Loss: 96.715080
Train Epoch: 7 [51200/60000 (85%)]	Loss: 97.578598
Train Epoch: 7 [52480/60000 (87%)]	Loss: 89.517181
Train Epoch: 7 [53760/60000 (90%)]	Loss: 95.692429
Train Epoch: 7 [55040/60000 (92%)]	Loss: 91.053009
Train Epoch: 7 [56320/60000 (94%)]	Loss: 100.328758
Train Epoch: 7 [57600/60000 (96%)]	Loss: 95.956688
Train Epoch: 7 [58880/60000 (98%)]	Loss: 91.104614
====> Epoch: 7 Average loss: 94.2428
====> Test set loss: 95.5679
Train Epoch: 8 [0/60000 (0%)]	Loss: 95.192459
Train Epoch: 8 [1280/60000 (2%)]	Loss: 93.376457
Train Epoch: 8 [2560/60000 (4%)]	Loss: 97.822693
Train Epoch: 8 [3840/60000 (6%)]	Loss: 92.193558
Train Epoch: 8 [5120/60000 (9%)]	Loss: 99.891571
Train Epoch: 8 [6400/60000 

In [16]:
num_gen = 16
with torch.no_grad():
    z = torch.randn(num_gen, ZDIMS).to(device)
    #y = torch.randint(0, num_classes, (64, 1)).to(dtype=torch.long)
    y = torch.ones(num_gen, 1).to(dtype=torch.long) * 3

    y = idx2onehot(y).to(device, dtype=z.dtype)
    z = torch.cat((z, y), dim=1)

    sample = model.decode(z).cpu()
    save_image(sample.view(num_gen, 1, 28, 28), 'results/sample_' + str(epoch) + '.png')